### Helpers

In [1]:
#import libraries
import cv2
import numpy as np

In [2]:
#input image
image = cv2.imread('.\\input images\\WGT1021-508.jpg', cv2.IMREAD_GRAYSCALE)
#image = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)
#dim = 512, 1024

## De-noising methods

#### Gaussian blur

In [12]:
kernel_size = (7, 7) #kernel size
denoised_image = cv2.GaussianBlur(image, kernel_size, 0) #apply Gaus. blur


In [13]:
#save the output
#cv2.imwrite('.\\output images\\gaussian_blurr.png', denoised_image) #save img
cv2.imwrite('output.png', denoised_image) #save img

True

#### Binary thresh-holding

In [25]:
#threshhold value
threshold = 128
    #pixels with value less than this will be turned to black
    #others to white

#apply threshholding
_, output = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)

#save the output image
cv2.imwrite('.\\output images\\threshhold_128.png', output) 

True

#### Filtering

In [15]:
#kernel size
kernel_size = 3

# median filter
denoised_image = cv2.medianBlur(image, kernel_size)


In [16]:
# save the output
cv2.imwrite('.\\output images\\median_filter.png', denoised_image)

True

#### Non-local Means de-noising

In [29]:
# filter strength 
h = 10 #higher value = stronger de-noising

# Apply Non-Local Means denoising
denoised_image = cv2.fastNlMeansDenoising(image, None, h=h)

# Save the output
cv2.imwrite('.\\output images\\NLM_10.jpg', denoised_image)


True

#### Adaptive NLM

In [28]:
######## adaptive NLM de-noising ########

h = 10
search_window = 100
patch_size = 7

# Apply adaptive Non-Local Means denoising
denoised_image = cv2.fastNlMeansDenoising(image, None, h=h, templateWindowSize=patch_size, searchWindowSize=search_window)


In [19]:
# Save the output
cv2.imwrite('.\\output images\\adaptive_NLM.jpg', denoised_image)

True

## Contrast enhancement

#### Histogram equalisation

In [38]:
##### contrast enhancement ########
# histogram equilisation technique
equalized_image = cv2.equalizeHist(denoised_image)

# Save the equalized image
#cv2.imwrite('.\\equalized_image.png', equalized_image)


In [39]:
# Save the output
cv2.imwrite('.\\output images\\adaptive_equalized.jpg', equalized_image)


True

#### CLAHE - (Contrast Limited Adaptive Histogram Equalization)

In [26]:
# Create a CLAHE (Contrast Limited Adaptive Histogram Equalization) object
clip = 2.5   #strength of contrast
grid = (5,7) #size of local region
clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=grid)

# Apply CLAHE to enhance local contrast
enhanced_image = clahe.apply(denoised_image)


In [27]:
### save the output ###
# Save the enhanced image
cv2.imwrite('.\\output images\\adaptiveNLM_CLAHE.jpg', enhanced_image)


True

#### Unsharp masking

In [37]:
# Apply Unsharp Masking to enhance local contrast (adjust parameters as needed)
kernel_size = (5, 5) #for gaussian blurr

alpha = 2.0 # Controls the amount of enhancement
beta = 10   # Controls the brightness

# Apply Gaussian blur to create the blurred version of the image
blurred = cv2.GaussianBlur(denoised_image, kernel_size, 0)

# Calculate the Unsharp Mask
unsharp_mask = cv2.addWeighted(denoised_image, alpha, blurred, -alpha, beta)

# Save the enhanced image
cv2.imwrite('.\\enhanced_image_UnsharpMask.png', unsharp_mask)


True

## Custom de-noising

In [ ]:
def denoise(img, threshold, box):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = img.copy()

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # denoise the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = patch[patch <= threshold]*(0.8)
                # make the non-black pixels brighter
                patch[patch > threshold] = avg*(1.1)
            
            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result